In [1]:
import argparse
import roypy
import time
import queue
from sample_camera_info import print_camera_info
from roypy_sample_utils import CameraOpener, add_camera_opener_options
#from roypy_platform_utils import PlatformHelper

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import os.path

try:
    import roypycy
except ImportError:
    print("Pico Flexx backend requirements (roypycy) not installed properly")
    raise


            

In [2]:
class MyListener(roypy.IDepthDataListener):
    def __init__(self, q):
        super(MyListener, self).__init__()
        self.queue = q

    def onNewData(self, data):
        grayvalues = []
        values = roypycy.get_backend_data(data)
        grayvalues = values.grayValue
        
        p = grayvalues.reshape (-1, data.width)        
        self.queue.put(p)
            

    def paint (self, name, data):
        """Called in the main thread, with data containing one of the items that was added to the
        queue in onNewData.
        """
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.imshow(name, data)
        cv2.waitKey(1)

In [3]:
def grayvalue_scale(img):
    grayimg_int8 = cv2.convertScaleAbs(img, alpha=(255.0/65535.0))
    scale = 255//grayimg_int8.max()
    grayimg = grayimg_int8*scale
    return grayimg

In [4]:
def storeimg(img, i):
    TARGETDIR = './depth_chessboard_data'
    if not os.path.exists(TARGETDIR):
        os.mkdir(TARGETDIR)
        
    cv2.imwrite(TARGETDIR + '/' + str(i).zfill(2) + '.png', img)
    
def storeRGBimg(img, i):
    TARGETDIR = './camera_chessboard_data'
    if not os.path.exists(TARGETDIR):
        os.mkdir(TARGETDIR)
        
    cv2.imwrite(TARGETDIR + '/' + str(i).zfill(2) + '.png', img)

In [5]:
def cailbration_data(image, objpoints, imgpoints, name):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    # (8,6) is for the given testing images.
    # If you use the another data (e.g. pictures you take by your smartphone), 
    # you need to set the corresponding numbers.
    corner_x = 8
    corner_y = 7
    objp = np.zeros((corner_x*corner_y,3), np.float32)
    objp[:,:2] = np.mgrid[0:corner_x, 0:corner_y].T.reshape(-1,2)
    
    #Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(image, (corner_x,corner_y), None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        newImage = image.copy()
        cv2.drawChessboardCorners(newImage, (corner_x,corner_y), corners, ret)
        
        #show image
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.imshow(name, newImage)
        cv2.waitKey(1)
        
    return ret, objpoints, imgpoints

def ceilbration(objpoints, imgpoints, img_size):
    print('Camera calibration...')
    # You need to comment these functions and write your calibration function from scratch.
    # Notice that rvecs is rotation vector, not the rotation matrix, and tvecs is translation vector.
    # In practice, you'll derive extrinsics matrixes directly. The shape must be [pts_num,3,4], and use them to plot.
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None,None)
    Vr = np.array(rvecs)
    Tr = np.array(tvecs)
    extrinsics = np.concatenate((Vr, Tr), axis=1).reshape(-1,6)
    print('intrinsics mtx :\n', mtx)
    #print('extrinsics :\n',extrinsics)
    #print('Tr :\n', Tr)
    
    return mtx, dist, rvecs, tvecs


In [6]:
def main ():
    parser = argparse.ArgumentParser (usage = __doc__)
    add_camera_opener_options (parser)
    parser.add_argument ("--seconds", type=int, default=15, help="duration to capture data")
    #options = parser.parse_args(args=['--rrf', '0108.rrf','--seconds', '5'])
    options = parser.parse_args(args=['--seconds', '500'])

    opener = CameraOpener (options)
    cam = opener.open_camera ()
    
    Replay = True
    if(Replay):
        cam.setUseCase('MODE_5_35FPS_600')

    print_camera_info (cam)
    print("isConnected", cam.isConnected())
    print("getFrameRate", cam.getFrameRate())
    print("UseCase",cam.getCurrentUseCase())

    # we will use this queue to synchronize the callback with the main
    # thread, as drawing should happen in the main thread
    q = queue.LifoQueue()
    l = MyListener(q)
    cam.registerDataListener(l)
    cam.startCapture()
    # create a loop that will run for a time (default 15 seconds)
    objpoints, imgpoints, img_size = process_event_queue (q, l, options.seconds)
    cam.stopCapture()
    
    cv2.destroyAllWindows()
    
    mtx, dist, rvecs, tvecs = ceilbration(objpoints, imgpoints, img_size)

# create a loop that will run for the given amount of time
def process_event_queue (q, painter, seconds):
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    img_size = (171, 224)
    num_of_img = 0
    Store = True
    Both = True
    
    t_end = time.time() + seconds
    
    if Both :
        RGB_objpoints = [] # 3d points in real world space
        RGB_imgpoints = [] # 2d points in image plane.
        capture = cv2.VideoCapture(0)
        capture.open(1 + cv2.CAP_DSHOW)
        capture.set(3, 1280) # set the resolution
        capture.set(4, 720)
        capture.set(cv2.CAP_PROP_AUTOFOCUS, 0) # turn the autofocus off  #0204 尚未測試
    else:
        RGB_ret = True

    while time.time() < t_end:
        try:
            # try to retrieve an item from the queue.
            # this will block until an item can be retrieved
            # or the timeout of 1 second is hit
            grayimg = q.get(True, 1)
            
#             ret, frame = capture.read()
#             gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        except queue.Empty:
            # this will be thrown when the timeout is hit
            break
        else:
#             grayimg = grayvalue_scale(grayimg)
            grayimg = cv2.convertScaleAbs(grayimg)
            
            #press space and collate data
            if cv2.waitKey(10) & 0xFF == ord(' '):
                ret, objpoints, imgpoints = cailbration_data(grayimg, objpoints, imgpoints, 'Chessboard')
                if Both :
                    ret, frame = capture.read()
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    RGB_ret, RGB_objpoints, RGB_imgpoints = cailbration_data(gray, RGB_objpoints, RGB_imgpoints, 'RGB_Chessboard')
                if Store and ret and RGB_ret:
                    num_of_img = num_of_img + 1
                    storeimg(grayimg, num_of_img)
                    if Both :
                        storeRGBimg(frame, num_of_img)
            img_size = (grayimg.shape[1], grayimg.shape[0])
            
            painter.paint ('grayvalues',grayimg)
#             painter.paint ('gray',gray)
            
            if num_of_img >= 5 or cv2.waitKey(10) & 0xFF == ord('q'):
                break
                    
    return objpoints, imgpoints, img_size

In [9]:
main()

Number of cameras connected:  1
        Camera information
Type:            PICOFLEXX
Width:           224
Height:          171
Operation modes: 10
    MODE_9_5FPS_2000
    MODE_9_10FPS_1000
    MODE_9_15FPS_700
    MODE_9_25FPS_450
    MODE_5_35FPS_600
    MODE_5_45FPS_500
    MODE_MIXED_30_5
        this operation mode has 2 streams
    MODE_MIXED_50_5
        this operation mode has 2 streams
    Low_Noise_Extended
    Fast_Acquisition
Lens parameters: 9
    ('cx', 118.28559112548828)
    ('cy', 87.74105072021484)
    ('fx', 213.8031768798828)
    ('fy', 213.8031768798828)
    ('k1', 0.4155448377132416)
    ('k2', -4.7316107749938965)
    ('k3', 8.45906925201416)
    ('p1', 7.605663946304829e-16)
    ('p2', 4.939198934392371e-16)
CameraInfo items: 8
    ('BRIDGE_TYPE', 'Enclustra')
    ('MODULE_IDENTIFIER', '00000000')
    ('MODULE_IDENTIFIER_HASH', '558161692')
    ('MODULE_SERIAL', '0')
    ('MODULE_SUFFIX', '')
    ('IMAGER', 'M2450_A12_AIO')
    ('PROCESSING_NAME', 'Spectre')
  